#### _Programming Exercise 2.4.1:_ Compute a taptweak from a taptree

In [ ]:
TAPSCRIPT_VER = bytes([0xc0])  # See tapscript chapter for more details.
internal_pubkey = ECPubKey()
internal_pubkey.set(bytes.fromhex('03af455f4989d122e9185f8c351dbaecd13adca3eef8a9d38ef8ffed6867e342e3'))

# Derive pay-to-pubkey scripts
privkeyA, pubkeyA = generate_bip340_key_pair()
privkeyB, pubkeyB = generate_bip340_key_pair()
privkeyC, pubkeyC = generate_bip340_key_pair()
scriptA = CScript([pubkeyA.get_bytes(), OP_CHECKSIG])
scriptB = CScript([pubkeyB.get_bytes(), OP_CHECKSIG])
scriptC = CScript([pubkeyC.get_bytes(), OP_CHECKSIG])

# Method: Returns tapbranch hash. Child hashes are lexographically sorted and then concatenated.
# l: tagged hash of left child
# r: tagged hash of right child
def tapbranch_hash(l, r):
    return tagged_hash("TapBranch", b''.join(sorted([l,r])))

# 1) Compute TapLeaves A, B and C
# Method: ser_string(data) is a function which adds compactsize to input data.
hash_inputA = TAPSCRIPT_VER + ser_string(scriptA)
hash_inputB = TAPSCRIPT_VER + ser_string(scriptB)
hash_inputC = TAPSCRIPT_VER + ser_string(scriptC)
taggedhash_leafA = tagged_hash("TapLeaf", hash_inputA)
taggedhash_leafB = tagged_hash("TapLeaf", hash_inputB)
taggedhash_leafC = tagged_hash("TapLeaf", hash_inputC)

# 2) Compute Internal node TapBranch AB
# Method: use tapbranch_hash() function
internal_nodeAB = tapbranch_hash(taggedhash_leafA, taggedhash_leafB)

# 3) Compute TapTweak
rootABC = tapbranch_hash(internal_nodeAB, taggedhash_leafC)
taptweak = tagged_hash("TapTweak", internal_pubkey.get_bytes() + rootABC)
print("TapTweak:", taptweak.hex())

# 4) Derive the segwit output address
taproot_pubkey_b = internal_pubkey.tweak_add(taptweak).get_bytes()
segwit_address = program_to_witness(1, taproot_pubkey_b)
print('Segwit address:', segwit_address)

#### _Programming Exercise 2.4.5_ - Constructing a taproot output from a taptree

In [ ]:
# Generate key pairs for internal pubkey and pay-to-pubkey tapscripts
privkey_internal, pubkey_internal = generate_bip340_key_pair()

privkeyA, pubkeyA = generate_bip340_key_pair()
privkeyB, pubkeyB = generate_bip340_key_pair()
privkeyC, pubkeyC = generate_bip340_key_pair()
privkeyD, pubkeyD = generate_bip340_key_pair()

# Construct Pay-to-Pubkey TapLeaves and Taptree.
TapLeafA = TapLeaf().construct_pk(pubkeyA)
TapLeafB = TapLeaf().construct_pk(pubkeyB)
TapLeafC = TapLeaf().construct_pk(pubkeyC)
TapLeafD = TapLeaf().construct_pk(pubkeyD)

# Create a Taptree with tapleaves and huffman constructor.
# Method: TapTree.huffman_constructor(tuple_list)
taptree = TapTree(key=pubkey_internal)
taptree.huffman_constructor([(1, TapLeafA), (1, TapLeafB), (1, TapLeafC), (1, TapLeafD)])

# Generate taproot tree with the `construct()` method, then use the taproot bytes to create a segwit address
taproot_script, tweak, control_map = taptree.construct()
taproot_pubkey = pubkey_internal.tweak_add(tweak) 
program = taproot_pubkey.get_bytes()
address = program_to_witness(1, program)
print("Address: {}".format(address))

#### _Programming Exercise 2.4.8:_ Sign the transaction for `TapLeafA` 

In [ ]:
# Generate the taproot signature hash for signing
sighashA = TaprootSignatureHash(spending_tx,
                               [tx.vout[0]],
                               SIGHASH_ALL_TAPROOT,
                               input_index=0,
                               scriptpath=True,
                               script=TapLeafA.script)

signatureA = privkeyA.sign_schnorr(sighashA)

print("Signature for TapLeafA: {}\n".format(signatureA.hex()))

#### _Programming Exercise  2.4.9:_ Construct the witness, add it to the transaction and verify mempool acceptance

In [ ]:
# Add witness to transaction
# Tip: Witness stack for script path - [satisfying elements for tapscript] [TapLeaf.script] [controlblock]
# Tip: Controlblock for a tapscript in control_map[TapLeaf.script]
witness_elements = [signatureA, TapLeafA.script, control_map[TapLeafA.script]]
spending_tx.wit.vtxinwit.append(CTxInWitness(witness_elements))

# Test mempool acceptance
assert node.test_transaction(spending_tx)
print("Success!")